In [1]:
import pandas as pd
import countryinfo as cf # Source: https://gist.github.com/canfixit/1662664

In [24]:
countries = cf.countries
eu_codes = []
eu_names = []
iso_name = {}
name_iso = {}
exclusion_list = [
    'San Marino', 'Russia', 'Vatican City', 'Monaco','Macedonia', 'Bosnia and Herzegovina', 
    'Serbia','Montenegro', 'Albania' ,'Moldova', 'Belarus', 'Andorra','Ukraine'
]
print("Excluded list: ")
for co in countries:
    
    # Excluding Russia, Monacco, San Marino and Vatican City bc. lack of data
    # As well as other europeean countries not suitable for the stakeholder
    if co['name'] in exclusion_list:
        print(co['name'])
        continue
        
    if co['continent'].lower() == 'europe' or co['name'] == "Cyprus":
        name = co['name']
        if name == 'Kingdom of the Netherlands':
            name = "Netherlands"

        iso = co['code']
        eu_codes.append(iso)
        eu_names.append(name)
        iso_name[iso] = name
        name_iso[name] = iso
        
tot_num_c = len(eu_codes)

print("Number of countries: ", tot_num_c)
eu_names

Excluded list: 
Andorra
Albania
Belarus
Macedonia
Russia
San Marino
Ukraine
Vatican City
Bosnia and Herzegovina
Moldova
Monaco
Montenegro
Serbia
Number of countries:  32


['Austria',
 'Belgium',
 'Bulgaria',
 'Cyprus',
 'Czech Republic',
 'Germany',
 'Denmark',
 'Estonia',
 'Finland',
 'France',
 'Greece',
 'Hungary',
 'Republic of Ireland',
 'Iceland',
 'Italy',
 'Liechtenstein',
 'Lithuania',
 'Luxembourg',
 'Latvia',
 'Malta',
 'Netherlands',
 'Norway',
 'Poland',
 'Portugal',
 'Romania',
 'Sweden',
 'Slovenia',
 'Slovakia',
 'Croatia',
 'Spain',
 'Switzerland',
 'United Kingdom']

In [25]:
# Main dataframe
df_main = None
# Time period from 2012 to 2022
time_period = list(range(1990, 2022))

## Temperature Offset
#### - Compared to pre-industrial levels

In [26]:
df_temp_change = pd.read_csv("data/Annual_Surface_Temperature_Change.csv")

df_temp_change = df_temp_change[df_temp_change['ISO2'].isin(eu_codes)]

df_temp_change = df_temp_change.drop(['Unit','Country', 'CTS_Code', 'Indicator', 'Source', 'CTS_Name', 'ObjectId', 'CTS_Full_Descriptor'], axis = 1, inplace=False)
df_temp_change.columns = [str(x).split("F")[-1] for x in df_temp_change.columns]

df_temp_change.columns= df_temp_change.columns.str.lower()
df_temp_change = pd.melt(df_temp_change, id_vars=["iso2", 'iso3'], var_name = ['year'], value_name = 'temp_change C')
df_temp_change["year"] = pd.to_numeric(df_temp_change["year"])

df_temp_change.sort_values(by=["iso3", 'year'], inplace=True)
df_temp_change.columns = ['ISO2', 'ISO3', 'year', 'Temperature Change C']
df_temp_change.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1952 entries, 0 to 1949
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ISO2                  1952 non-null   object 
 1   ISO3                  1952 non-null   object 
 2   year                  1952 non-null   int64  
 3   Temperature Change C  1655 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 76.2+ KB


## CO2 Emissions - Annual 

In [27]:
df_co2 = pd.read_csv("data/annual-co2-emissions-per-country.csv")
df_co2.columns = df_co2.columns.str.lower()
df_co2.columns = ['country', 'ISO3', 'year', 'annual co2 emmisions']
df_co2 = df_co2[df_co2['ISO3'].isin(set(df_temp_change['ISO3']))]
df_co2['y/y Change in Emmision (%)'] = df_co2['annual co2 emmisions'].pct_change()

df_co2["year"] = pd.to_numeric(df_co2["year"])
df_co2 = df_co2[['ISO3', 'year', 'annual co2 emmisions', 'y/y Change in Emmision (%)']]
df_co2.rename(columns={'annual co2 emmisions' : 'annual t co2 emmisions'}, inplace = True)
print("All countries: ", len((set(eu_codes) - set(df_co2['ISO3']))) == 0)
df_co2.info()

All countries:  False
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5253 entries, 2563 to 29726
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ISO3                        5253 non-null   object 
 1   year                        5253 non-null   int64  
 2   annual t co2 emmisions      5253 non-null   float64
 3   y/y Change in Emmision (%)  5066 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 205.2+ KB


## GDP

In [28]:
df_gdp = pd.read_csv("data/GDP.csv",delimiter=";")
#Drop last column, as it is no values
df_gdp = df_gdp.iloc[: , :-1]

df_gdp = df_gdp[df_gdp['Country Code'].isin(set(df_temp_change['ISO3']))]
df_gdp.drop(['Country Name', 'Indicator Name', 'Indicator Code'], axis = 1, inplace = True)
df_gdp.columns = df_gdp.columns.str.lower()
df_gdp.rename(columns = {'country code' : "ISO3"}, inplace=True)

df_gdp = pd.melt(df_gdp, id_vars=["ISO3"], var_name = ['year'], value_name = 'GDP US$')
df_gdp["year"] = pd.to_numeric(df_gdp["year"])

print("All countries: ", len((set(eu_codes) - set(df_gdp['ISO3']))) == 0)

df_gdp.info()

All countries:  False
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1984 entries, 0 to 1983
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ISO3     1984 non-null   object 
 1   year     1984 non-null   int64  
 2   GDP US$  1578 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 46.6+ KB


## Green Bonds Annual Issued

In [29]:
# Green Bonds (in Billion $) Probably not inflation adjusted

df_green_bonds = pd.read_csv("data/Green_Bonds.csv")

df_green_bonds = df_green_bonds[df_green_bonds['ISO3'].isin(set(df_temp_change['ISO3']))]

df_green_bonds.drop(['ISO2', 'Indicator', 'Source', 'CTS_Code', 'CTS_Full_Descriptor', 'Type_of_Issuer', 'Use_of_Proceed', 'Principal_Currency'],axis = 1, inplace=True)
df_green_bonds.columns = df_green_bonds.columns.str.lower()
df_green_bonds.columns = [str(x).split("f")[-1] for x in df_green_bonds.columns]

# Only keep cts_name equals Green Bonds Issuances, as we can calculate total later (cts_name == Green Bonds)
df_green_bonds = df_green_bonds.loc[df_green_bonds['cts_name'] == 'Green Bonds Issuances'] 
df_green_bonds.drop(['cts_name', 'unit', 'objectid', 'country'],axis = 1, inplace=True)

df_green_bonds.rename(columns = {'iso3' : "ISO3"}, inplace=True)

df_green_bonds = pd.melt(df_green_bonds, id_vars=["ISO3"], var_name = ['year'], value_name = 'Green Bonds Issuance (Billion US$)')
df_green_bonds["year"] = pd.to_numeric(df_green_bonds["year"])
df_green_bonds.sort_values(by=["ISO3", 'year'], inplace=True)

print("Missing countries: ", (set(eu_codes) - set(df_green_bonds['ISO3'])))

df_green_bonds['Change Green Bonds'] = df_green_bonds['Green Bonds Issuance (Billion US$)'].pct_change()
df_green_bonds

Missing countries:  {'AT', 'LT', 'EE', 'FR', 'NL', 'CY', 'IT', 'BE', 'LU', 'CH', 'SI', 'IE', 'PL', 'IS', 'FI', 'NO', 'CZ', 'MT', 'GR', 'SK', 'PT', 'BG', 'HU', 'LV', 'HR', 'GB', 'DK', 'ES', 'DE', 'RO', 'LI', 'SE'}


,ISO3,year,Green Bonds Issuance (Billion US$),Change Green Bonds
0,AUT,1985,NaN,NaN
28,AUT,1986,NaN,NaN
56,AUT,1987,NaN,NaN
84,AUT,1990,NaN,NaN
112,AUT,1991,NaN,NaN
...,...,...,...,...
669,SWE,2017,4.696447,0.272868
697,SWE,2018,7.155104,0.523514
725,SWE,2019,13.382280,0.870312
753,SWE,2020,15.270920,0.141130


As we observe from the output, we are missing substantial data from some countries, as well as from many years, as green bonds issuance is quite new

## Population

In [30]:
# Population df

df_pop = pd.read_csv("data/population.csv")
df_pop = df_pop[df_pop['Country Code'].isin(set(df_temp_change['ISO3']))]
df_pop.columns = df_pop.columns.str.lower()
df_pop.columns = [str(x).split("[")[0] for x in df_pop.columns]
df_pop.drop(['series name', 'series code', 'country name'] ,axis=1, inplace = True)
df_pop.rename(columns = {'country code' : "ISO3"}, inplace=True)
df_pop = pd.melt(df_pop, id_vars=["ISO3"], var_name = ['year'], value_name = 'population')
df_pop["year"] = pd.to_numeric(df_pop["year"])
df_pop["population"] = pd.to_numeric(df_pop["population"])
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ISO3        1600 non-null   object
 1   year        1600 non-null   int64 
 2   population  1600 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 37.6+ KB


## Electric Vehicle Sales

BEVs are battery electric vehicles. 

PHEVs are plug-in hybrid electric vehicles. 

FCEVs are fuel cell electric vehicles. 

EVs refers to all electric vehicles (BEVs + PHEVs).

In [31]:
df_sales = pd.read_csv("data/IEA-EV-dataEV salesCarsHistorical.csv")


In [32]:
df_sales[df_sales['region'] == "Norway"].head()


,region,category,parameter,mode,powertrain,year,unit,value
397,Norway,Historical,EV sales,Cars,BEV,2010,sales,360
398,Norway,Historical,EV sales,Cars,BEV,2011,sales,2000
399,Norway,Historical,EV sales,Cars,BEV,2012,sales,3900
400,Norway,Historical,EV sales,Cars,PHEV,2012,sales,320
401,Norway,Historical,EV sales,Cars,PHEV,2013,sales,340


In [33]:

# one for BEVs and one for PHEV ??
df_bev_sales = df_sales[df_sales['powertrain'] == "BEV"]
df_phev_sales = df_sales[df_sales['powertrain'] == "PHEV"]

df_bev_sales = df_bev_sales[['region', 'year', 'value']]
df_phev_sales = df_phev_sales[['region', 'year', 'value']]

s = set(df_phev_sales['region'].unique())
s-set(eu_names)
before = set(eu_names) - s

df_bev_sales = df_bev_sales[df_bev_sales['region'].isin(eu_names)]
df_phev_sales = df_phev_sales[df_phev_sales['region'].isin(eu_names)]

s = set(df_phev_sales['region'].unique())
s-set(eu_names)
after = set(eu_names) - s
print(before - after)
df_bev_sales['region'].replace(name_iso, inplace = True)
df_phev_sales['region'].replace(name_iso, inplace = True)

df_bev_sales.rename(columns = {"region" : "ISO2", "value" : "Battery Electric Vehicles sales"}, inplace = True)
df_phev_sales.rename(columns = {"region" : "ISO2", "value" : "Plugin Hybrid Vehicles sales"}, inplace = True)

df_phev_sales

set()


,ISO2,year,Plugin Hybrid Vehicles sales
23,BE,2011,16
24,BE,2012,260
27,BE,2013,60
28,BE,2014,650
31,BE,2015,2200
...,...,...,...
584,GB,2017,34000
587,GB,2018,35000
588,GB,2019,37000
591,GB,2020,67000


In [34]:
df_ev_chargingP = pd.read_csv("data/IEA-EV-dataEV charging pointsHistorical.csv")
df_ev_chargingP

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV charging points,EV,Publicly available fast,2017,charging points,40.0
1,Australia,Historical,EV charging points,EV,Publicly available slow,2017,charging points,440.0
2,Australia,Historical,EV charging points,EV,Publicly available fast,2018,charging points,61.0
3,Australia,Historical,EV charging points,EV,Publicly available slow,2018,charging points,670.0
4,Australia,Historical,EV charging points,EV,Publicly available fast,2019,charging points,250.0
...,...,...,...,...,...,...,...,...
522,World,Historical,EV charging points,EV,Publicly available slow,2019,charging points,630000.0
523,World,Historical,EV charging points,EV,Publicly available fast,2020,charging points,390000.0
524,World,Historical,EV charging points,EV,Publicly available slow,2020,charging points,910000.0
525,World,Historical,EV charging points,EV,Publicly available fast,2021,charging points,560000.0


In [35]:
df_ev_chargingP = pd.read_csv("data/IEA-EV-dataEV charging pointsHistorical.csv")

s = set(df_ev_chargingP['region'].unique())
s-set(eu_names)
before = set(eu_names) - s

df_ev_chargingP = df_ev_chargingP[df_ev_chargingP['region'].isin(eu_names)]
df_ev_chargingP = df_ev_chargingP[['region', 'year', 'powertrain', 'value']]
df_ev_chargingP_fast = df_ev_chargingP[df_ev_chargingP['powertrain'] == 'Publicly available fast']
df_ev_chargingP_slow = df_ev_chargingP[df_ev_chargingP['powertrain'] == 'Publicly available slow']

s = set(df_ev_chargingP['region'].unique())
s-set(eu_names)
after = set(eu_names) - s
print(before - after)

df_ev_chargingP = df_ev_chargingP_fast.merge(
    df_ev_chargingP_slow, 
    left_on = ['region', 'year'], 
    right_on = ['region', 'year'],
    how = 'inner',
    suffixes=('_fast', '_slow')
)
df_ev_chargingP['total number of chargingpoints'] = df_ev_chargingP['value_fast'] + df_ev_chargingP['value_slow']
df_ev_chargingP = df_ev_chargingP[['region', 'year', 'value_fast', 'value_slow', 'total number of chargingpoints']]
df_ev_chargingP['region'].replace(name_iso, inplace = True)
df_ev_chargingP.rename(
    columns = {'value_fast': 'number of fast EV charginpoints', 
               'value_slow' : 'number of slow EV charginpoints',
               'region' : 'ISO2'},
                inplace = True)
df_ev_chargingP


set()


,ISO2,year,number of fast EV charginpoints,number of slow EV charginpoints,total number of chargingpoints
0,BE,2013,47.0,330.0,377.0
1,BE,2014,55.0,560.0,615.0
2,BE,2015,77.0,1300.0,1377.0
3,BE,2016,110.0,1500.0,1610.0
4,BE,2017,220.0,1500.0,1720.0
...,...,...,...,...,...
142,GB,2017,2200.0,13000.0,15200.0
143,GB,2018,2700.0,15000.0,17700.0
144,GB,2019,4700.0,22000.0,26700.0
145,GB,2020,6200.0,27000.0,33200.0


# Merging into one main dataframe
- With average and total as two seperate datasheets
- Writing all final DataFrames into a complete Excel file

In [36]:
df_main = df_temp_change

# Check if all conseccutive years are present
# Check if all years still in the dataset for every country

for c in df_main['ISO3'].unique():
    for_this_c = df_main[df_main['ISO3'] == c]
    all_years = (for_this_c['year'] == list(range(min(for_this_c['year']), max(for_this_c['year']) + 1))).all()
    if not all_years:
        print("Not Consecutive years included for: ", c)
        
print("Rest have all years included")

# Check if all countries in dataset
print("Number of countries before: ",len(df_main['ISO2'].unique()) == len(eu_codes))

# Merge instead of join, because join uses index, while merge could specify
# multiple columns
# Using left joint, since we want to hold all values already in the dataframe
# for instance the years, so all missing years in later merged dataframes
# will be set to Nans

# Merge all dataframes

dfs = [df_co2, df_gdp, df_green_bonds, df_pop]

for frame in dfs:
    
    df_main = df_main.merge(frame, left_on=['ISO3','year'], right_on = ['ISO3', 'year'], how = 'left')

# Merge EV-files on ISO2
dfs = [df_phev_sales, df_bev_sales, df_ev_chargingP]
for frame in dfs:
    df_main = df_main.merge(frame, left_on=['ISO2','year'], right_on = ['ISO2', 'year'], how = 'left')
    
# Insert country name
df_main['country'] = df_main['ISO2']
df_main['country'].replace(iso_name, inplace=True)


# Check if all countries still in the dataset
print("Number of countries after: ",len(df_main['country'].unique()) == len(eu_codes))

for c in df_main['ISO3'].unique():
    for_this_c = df_main[df_main['ISO3'] == c]
    all_years = (for_this_c['year'] == list(range(min(for_this_c['year']), max(for_this_c['year']) + 1))).all()
    if not all_years:
        print("Not Consecutive years included for: ", c)
        
print("Rest have all years included")

#f = df_main[df_main['country'] == 'United Kingdom']["annual t co2 emmisions"]
#f
f = df_main[df_main['country'] == 'Germany']["annual t co2 emmisions"]
df_main

Rest have all years included
Number of countries before:  True
Number of countries after:  True
Rest have all years included


,ISO2,ISO3,year,Temperature Change C,annual t co2 emmisions,y/y Change in Emmision (%),GDP US$,Green Bonds Issuance (Billion US$),Change Green Bonds,population,Plugin Hybrid Vehicles sales,Battery Electric Vehicles sales,number of fast EV charginpoints,number of slow EV charginpoints,total number of chargingpoints,country
0,AT,AUT,1961,1.031,31824046.0,0.033769,7.311750e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Austria
1,AT,AUT,1962,-0.624,33864984.0,0.064132,7.756110e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Austria
2,AT,AUT,1963,-0.729,36949060.0,0.091070,8.374175e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Austria
3,AT,AUT,1964,-0.371,38896436.0,0.052704,9.169984e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Austria
4,AT,AUT,1965,-0.884,38140550.0,-0.019433,9.994071e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Austria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,SE,SWE,2017,1.593,42704240.0,-0.016394,5.410187e+11,4.696447,0.272868,10057698.0,16000.0,4400.0,790.0,2600.0,3390.0,Sweden
1948,SE,SWE,2018,1.849,42094908.0,-0.014269,5.554554e+11,7.155104,0.523514,10175214.0,22000.0,7100.0,900.0,3200.0,4100.0,Sweden
1949,SE,SWE,2019,1.572,40982492.0,-0.026426,5.338795e+11,13.382280,0.870312,10278887.0,25000.0,16000.0,1000.0,4000.0,5000.0,Sweden
1950,SE,SWE,2020,2.953,36515104.0,-0.109007,5.414872e+11,15.270920,0.141130,10353442.0,66000.0,28000.0,1600.0,8800.0,10400.0,Sweden


In [37]:
df_main.describe()

,year,Temperature Change C,annual t co2 emmisions,y/y Change in Emmision (%),GDP US$,Green Bonds Issuance (Billion US$),Change Green Bonds,population,Plugin Hybrid Vehicles sales,Battery Electric Vehicles sales,number of fast EV charginpoints,number of slow EV charginpoints,total number of chargingpoints
count,1952.000000,1655.000000,1.923000e+03,1923.000000,1.562000e+03,156.000000,749.0,1.600000e+03,165.000000,187.000000,147.000000,147.000000,147.000000
mean,1991.000000,0.733279,1.292379e+08,0.011215,3.408779e+11,5.539923,inf,1.552739e+07,14693.315152,15062.786096,954.789116,8036.979592,8991.768707
std,17.611329,0.871814,2.013136e+08,0.073343,6.545307e+11,10.367159,NaN,2.099156e+07,35270.484443,38852.375977,1605.803316,12906.046446,13922.754161
min,1961.000000,-1.793000,1.419960e+05,-0.972617,9.009833e+07,0.000000,-1.0,2.216100e+04,1.000000,2.000000,1.000000,11.000000,22.000000
25%,1976.000000,0.044000,1.739022e+07,-0.026693,2.001223e+10,0.416132,0.0,2.762342e+06,340.000000,395.000000,78.000000,1100.000000,1219.500000
50%,1991.000000,0.753000,4.958335e+07,0.010317,8.503060e+10,1.571946,0.0,7.544770e+06,3100.000000,2200.000000,340.000000,2700.000000,3260.000000
75%,2006.000000,1.379500,1.258108e+08,0.049286,2.988262e+11,5.333590,0.0,1.202306e+07,16000.000000,11000.000000,1000.000000,8300.000000,9260.000000
max,2021.000000,3.595000,1.117888e+09,0.367647,4.223116e+12,71.700960,inf,8.316087e+07,330000.000000,360000.000000,9200.000000,83000.000000,85600.000000


In [38]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1952 entries, 0 to 1951
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ISO2                                1952 non-null   object 
 1   ISO3                                1952 non-null   object 
 2   year                                1952 non-null   int64  
 3   Temperature Change C                1655 non-null   float64
 4   annual t co2 emmisions              1923 non-null   float64
 5   y/y Change in Emmision (%)          1923 non-null   float64
 6   GDP US$                             1562 non-null   float64
 7   Green Bonds Issuance (Billion US$)  156 non-null    float64
 8   Change Green Bonds                  749 non-null    float64
 9   population                          1600 non-null   float64
 10  Plugin Hybrid Vehicles sales        165 non-null    float64
 11  Battery Electric Vehicles sales     187 non

## Average DataFrame

In [39]:
# Making a average DataFrame
df_average = df_main.groupby(["year"], as_index=False)
df_average = df_average.mean()
df_average


,year,Temperature Change C,annual t co2 emmisions,y/y Change in Emmision (%),GDP US$,Green Bonds Issuance (Billion US$),Change Green Bonds,population,Plugin Hybrid Vehicles sales,Battery Electric Vehicles sales,number of fast EV charginpoints,number of slow EV charginpoints,total number of chargingpoints
0,1961,0.907957,9.104080e+07,0.039306,1.799943e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1962,-0.219217,9.632505e+07,0.075410,1.963547e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1963,-0.481087,1.019603e+08,0.088141,2.166625e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1964,-0.044739,1.063450e+08,0.073750,2.398202e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1965,-0.516870,1.079438e+08,0.042073,2.612632e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2017,1.512188,1.123991e+08,0.017202,5.812301e+11,4.398360,0.750004,1.644860e+07,9490.000,8138.3125,642.4000,7690.133333,8332.533333
57,2018,2.005688,1.098124e+08,-0.014684,6.276566e+11,4.610240,9.726477,1.648726e+07,10917.500,11815.5000,875.0000,8596.000000,9471.000000
58,2019,1.936500,1.048245e+08,-0.036983,6.168287e+11,8.428476,1.952621,1.651112e+07,12311.875,21412.5000,1392.0000,11097.000000,12489.000000
59,2020,2.281813,9.459273e+07,-0.088455,6.191357e+11,8.403601,0.099287,1.653066e+07,37693.750,44086.2500,2087.4375,13908.125000,15995.562500


## Total DataFrame

In [40]:
# Making total DataFrame
df_total = df_main.groupby(["year"], as_index=False)
df_total = df_total.sum()
df_total['y/y Change in Emmision (%)'] = df_total['annual t co2 emmisions'].pct_change()
df_total['Change Green Bonds'] = df_total['Green Bonds Issuance (Billion US$)'].pct_change()
df_total['Change Green Bonds'] = df_total['Change Green Bonds'].astype(float)
df_total['BEV Sales Change'] = df_total['Battery Electric Vehicles sales'].pct_change()
df_total

,year,Temperature Change C,annual t co2 emmisions,y/y Change in Emmision (%),GDP US$,Green Bonds Issuance (Billion US$),Change Green Bonds,population,Plugin Hybrid Vehicles sales,Battery Electric Vehicles sales,number of fast EV charginpoints,number of slow EV charginpoints,total number of chargingpoints,BEV Sales Change
0,1961,20.883,2.822265e+09,NaN,2.879909e+11,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,1962,-5.042,2.986077e+09,0.058043,3.141676e+11,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,1963,-11.065,3.160770e+09,0.058503,3.466600e+11,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,1964,-1.029,3.296694e+09,0.043003,3.837123e+11,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,1965,-11.888,3.346259e+09,0.015035,4.180211e+11,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2017,48.390,3.596772e+09,0.002791,1.859936e+13,65.975395,1.117861,526355216.0,151840.0,130213.0,9636.0,115352.0,124988.0,0.495309
57,2018,64.182,3.513997e+09,-0.023014,2.008501e+13,82.984313,0.257807,527592341.0,174680.0,189048.0,13125.0,128940.0,142065.0,0.451837
58,2019,61.968,3.354383e+09,-0.045423,1.973852e+13,151.712572,0.828208,528355897.0,196990.0,342600.0,22272.0,177552.0,199824.0,0.812238
59,2020,73.018,3.026967e+09,-0.097608,1.919321e+13,168.072022,0.107832,528981129.0,603100.0,705380.0,33399.0,222530.0,255929.0,1.058903


In [41]:

df_main = df_main[df_main['year'].isin(set(time_period))]
df_main = df_main.astype({'year' : 'int32'}, copy = True)
df_main.year = pd.to_datetime(df_main.year, format='%Y')

df_average = df_average[df_average['year'].isin(set(time_period))]
df_average = df_average.astype({'year' : 'int32'}, copy = True)
df_average.year = pd.to_datetime(df_average.year, format='%Y')

df_total  = df_total[df_total['year'].isin(set(time_period))]
df_total = df_total.astype({'year' : 'int32'}, copy = True)
df_total.year = pd.to_datetime(df_total.year, format='%Y')


df_main

,ISO2,ISO3,year,Temperature Change C,annual t co2 emmisions,y/y Change in Emmision (%),GDP US$,Green Bonds Issuance (Billion US$),Change Green Bonds,population,Plugin Hybrid Vehicles sales,Battery Electric Vehicles sales,number of fast EV charginpoints,number of slow EV charginpoints,total number of chargingpoints,country
29,AT,AUT,1990-01-01,1.268,62145250.0,0.157347,1.664634e+11,NaN,NaN,7677850.0,NaN,NaN,NaN,NaN,NaN,Austria
30,AT,AUT,1991-01-01,0.130,65743936.0,0.057908,1.737942e+11,NaN,NaN,7754891.0,NaN,NaN,NaN,NaN,NaN,Austria
31,AT,AUT,1992-01-01,1.250,60243908.0,-0.083658,1.950781e+11,NaN,NaN,7840709.0,NaN,NaN,NaN,NaN,NaN,Austria
32,AT,AUT,1993-01-01,0.529,60670572.0,0.007082,1.903797e+11,NaN,NaN,7905633.0,NaN,NaN,NaN,NaN,NaN,Austria
33,AT,AUT,1994-01-01,1.962,61045756.0,0.006184,2.035352e+11,NaN,NaN,7936118.0,NaN,NaN,NaN,NaN,NaN,Austria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,SE,SWE,2017-01-01,1.593,42704240.0,-0.016394,5.410187e+11,4.696447,0.272868,10057698.0,16000.0,4400.0,790.0,2600.0,3390.0,Sweden
1948,SE,SWE,2018-01-01,1.849,42094908.0,-0.014269,5.554554e+11,7.155104,0.523514,10175214.0,22000.0,7100.0,900.0,3200.0,4100.0,Sweden
1949,SE,SWE,2019-01-01,1.572,40982492.0,-0.026426,5.338795e+11,13.382280,0.870312,10278887.0,25000.0,16000.0,1000.0,4000.0,5000.0,Sweden
1950,SE,SWE,2020-01-01,2.953,36515104.0,-0.109007,5.414872e+11,15.270920,0.141130,10353442.0,66000.0,28000.0,1600.0,8800.0,10400.0,Sweden


In [42]:
first_value = df_total['annual t co2 emmisions'].tolist()[0]
df_total['Emission change since 1990'] = (df_total['annual t co2 emmisions'] - first_value)/first_value
df_total

,year,Temperature Change C,annual t co2 emmisions,y/y Change in Emmision (%),GDP US$,Green Bonds Issuance (Billion US$),Change Green Bonds,population,Plugin Hybrid Vehicles sales,Battery Electric Vehicles sales,number of fast EV charginpoints,number of slow EV charginpoints,total number of chargingpoints,BEV Sales Change,Emission change since 1990
29,1990-01-01,27.166,4.549109e+09,0.012855,7.903037e+12,0.000000,NaN,488966193.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000000
30,1991-01-01,7.386,4.494420e+09,-0.012022,8.227680e+12,0.029179,inf,490504090.0,0.0,0.0,0.0,0.0,0.0,NaN,-0.012022
31,1992-01-01,21.732,4.351142e+09,-0.031879,8.950102e+12,0.143546,3.919436,491996666.0,0.0,0.0,0.0,0.0,0.0,NaN,-0.043518
32,1993-01-01,7.359,4.268875e+09,-0.018907,8.172726e+12,0.185650,0.293318,493603706.0,0.0,0.0,0.0,0.0,0.0,NaN,-0.061602
33,1994-01-01,34.283,4.245703e+09,-0.005428,8.685738e+12,0.109745,-0.408859,494891715.0,0.0,0.0,0.0,0.0,0.0,NaN,-0.066696
34,1995-01-01,28.153,4.287476e+09,0.009839,1.015716e+13,0.000000,-1.000000,495920609.0,0.0,0.0,0.0,0.0,0.0,NaN,-0.057513
35,1996-01-01,-4.137,4.395844e+09,0.025275,1.036695e+13,0.000000,NaN,496817267.0,0.0,0.0,0.0,0.0,0.0,NaN,-0.033691
36,1997-01-01,17.012,4.305054e+09,-0.020654,9.759677e+12,0.000000,NaN,497652023.0,0.0,0.0,0.0,0.0,0.0,NaN,-0.053649
37,1998-01-01,26.578,4.303590e+09,-0.000340,1.009221e+13,0.000000,NaN,498444826.0,0.0,0.0,0.0,0.0,0.0,NaN,-0.053971
38,1999-01-01,31.208,4.242333e+09,-0.014234,1.008396e+13,0.048174,inf,499414002.0,0.0,0.0,0.0,0.0,0.0,NaN,-0.067436


In [43]:
## Write all DataFrames to excel

writer = pd.ExcelWriter('data/main.xlsx', engine='xlsxwriter')

df_main.to_excel(writer, sheet_name = "main")
df_average.to_excel(writer, sheet_name = "average")
df_total.to_excel(writer, sheet_name = "total")

writer.save()

In [44]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1024 entries, 29 to 1951
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ISO2                                1024 non-null   object        
 1   ISO3                                1024 non-null   object        
 2   year                                1024 non-null   datetime64[ns]
 3   Temperature Change C                988 non-null    float64       
 4   annual t co2 emmisions              1024 non-null   float64       
 5   y/y Change in Emmision (%)          1024 non-null   float64       
 6   GDP US$                             996 non-null    float64       
 7   Green Bonds Issuance (Billion US$)  153 non-null    float64       
 8   Change Green Bonds                  668 non-null    float64       
 9   population                          1024 non-null   float64       
 10  Plugin Hybrid Vehicles 